In [1]:
import pygame
import random
import os

pygame.init()
pygame.mixer.init()  # Initialize the mixer for sound

WIDTH, HEIGHT = 800, 600
win = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Space Invaders - Enemy AI")

clock = pygame.time.Clock()
FPS = 45

# Colors
WHITE = (255, 255, 255)
RED = (255, 0, 0)
ORANGE = (255, 165, 0)
GRAY = (100, 100, 100)

SHIP_WIDTH, SHIP_HEIGHT = 64, 64
ENEMY_SIZE = 64
BOSS_SIZE = 128
BULLET_WIDTH, BULLET_HEIGHT = 16, 40

# Load images
spaceship_img = pygame.transform.scale(pygame.image.load("spaceship.png"), (SHIP_WIDTH, SHIP_HEIGHT))
enemy_img = pygame.transform.scale(pygame.image.load("enemy.png"), (ENEMY_SIZE, ENEMY_SIZE))
boss_img = pygame.transform.scale(pygame.image.load("boss.png"), (BOSS_SIZE, BOSS_SIZE))
bullet_img = pygame.transform.scale(pygame.image.load("bullet.png"), (BULLET_WIDTH, BULLET_HEIGHT))
background_img = pygame.transform.scale(pygame.image.load("background.jpg"), (WIDTH, HEIGHT))

# Load sounds
shoot_sound = pygame.mixer.Sound("shoot.wav")
explosion_sound = pygame.mixer.Sound("explosion.wav")
hit_sound = pygame.mixer.Sound("hit.wav")
start_sound = pygame.mixer.Sound("start.mp3")
background_music = pygame.mixer.Sound("background_music.wav")

# Fonts
font = pygame.font.Font(None, 36)
title_font = pygame.font.Font(None, 72)

# Game states
MENU = "menu"
PLAYING = "playing"
GAME_OVER = "game_over"
HIGH_SCORE = "high_score"
game_state = MENU

# High score file
HIGH_SCORE_FILE = "high_score.txt"

def load_high_score():
    if os.path.exists(HIGH_SCORE_FILE):
        with open(HIGH_SCORE_FILE, "r") as f:
            return int(f.read())
    return 0

def save_high_score(score):
    with open(HIGH_SCORE_FILE, "w") as f:
        f.write(str(score))

class Button:
    def __init__(self, text, x, y, width, height, color, hover_color, text_color):
        self.text = text
        self.rect = pygame.Rect(x, y, width, height)
        self.color = color
        self.hover_color = hover_color
        self.text_color = text_color

    def draw(self):
        mouse_pos = pygame.mouse.get_pos()
        if self.rect.collidepoint(mouse_pos):
            pygame.draw.rect(win, self.hover_color, self.rect)
        else:
            pygame.draw.rect(win, self.color, self.rect)
        text_surface = font.render(self.text, True, self.text_color)
        text_rect = text_surface.get_rect(center=self.rect.center)
        win.blit(text_surface, text_rect)

    def is_clicked(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            if self.rect.collidepoint(event.pos):
                return True
        return False

class Spaceship:
    def __init__(self):
        self.x = WIDTH // 2 - SHIP_WIDTH // 2
        self.y = HEIGHT - SHIP_HEIGHT - 10
        self.speed = 6
        self.lives = 3

    def draw(self):
        win.blit(spaceship_img, (self.x, self.y))

    def move(self, keys):
        if keys[pygame.K_LEFT] and self.x > 0:
            self.x -= self.speed
        if keys[pygame.K_RIGHT] and self.x < WIDTH - SHIP_WIDTH:
            self.x += self.speed
        if keys[pygame.K_UP] and self.y > 0:
            self.y -= self.speed
        if keys[pygame.K_DOWN] and self.y < HEIGHT - SHIP_HEIGHT:
            self.y += self.speed

class Enemy:
    def __init__(self, x, y, speed_scale=1):
        self.x = x
        self.y = y
        self.speed_x = random.choice([-1, 1]) * random.uniform(0.5, 1.5) * speed_scale
        self.speed_y = random.uniform(0.2, 0.6) * speed_scale

    def move(self):
        self.x += self.speed_x
        self.y += self.speed_y
        if self.x <= -ENEMY_SIZE or self.x >= WIDTH:
            self.x = random.choice([-ENEMY_SIZE, WIDTH])
            self.y = random.randint(20, 150)

    def draw(self):
        win.blit(enemy_img, (self.x, self.y))

class Boss:
    def __init__(self):
        self.x = WIDTH // 2 - BOSS_SIZE // 2
        self.y = 30
        self.speed_x = random.choice([-1, 1]) * 2
        self.health = 20

    def move(self):
        self.x += self.speed_x
        if self.x <= 0 or self.x >= WIDTH - BOSS_SIZE:
            self.speed_x *= -1

    def draw(self):
        win.blit(boss_img, (self.x, self.y))
        pygame.draw.rect(win, RED, (self.x, self.y - 10, BOSS_SIZE, 5))
        pygame.draw.rect(win, (0, 255, 0), (self.x, self.y - 10, BOSS_SIZE * (self.health / 20), 5))

class Bullet:
    def __init__(self, x, y, speed):
        self.x = x
        self.y = y
        self.speed = speed

    def draw(self):
        win.blit(bullet_img, (self.x, self.y))

    def move(self):
        self.y += self.speed

# Initialize game variables
spaceship = Spaceship()
waves = 1
wave_enemy_count = 10  # Fixed typo here
enemies = []
player_bullets = []
enemy_bullets = []
score = 0
boss = None
damage = 10
high_score = load_high_score()

# Buttons for menu
start_button = Button("Start Game", WIDTH//2 - 100, HEIGHT//2 - 100, 200, 50, GRAY, ORANGE, WHITE)
high_score_button = Button("High Score", WIDTH//2 - 100, HEIGHT//2, 200, 50, GRAY, ORANGE, WHITE)
exit_button = Button("Exit", WIDTH//2 - 100, HEIGHT//2 + 100, 200, 50, GRAY, ORANGE, WHITE)

# Buttons for game over
play_again_button = Button("Play Again", WIDTH//2 - 100, HEIGHT//2, 200, 50, GRAY, ORANGE, WHITE)
exit_game_over_button = Button("Exit", WIDTH//2 - 100, HEIGHT//2 + 100, 200, 50, GRAY, ORANGE, WHITE)

run = True
while run:
    clock.tick(FPS)
    win.blit(background_img, (0, 0))  # Draw background

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False

        if game_state == MENU:
            if start_button.is_clicked(event):
                game_state = PLAYING
                spaceship = Spaceship()
                waves = 1
                wave_enemy_count = 10
                enemies = [Enemy(random.choice([-ENEMY_SIZE, WIDTH]), random.randint(20, 150), waves) for _ in range(wave_enemy_count)]
                player_bullets = []
                enemy_bullets = []
                score = 0
                boss = None
                damage = 10
                start_sound.play()
                pygame.mixer.music.load("background_music.wav")
                pygame.mixer.music.play(-1)  # Loop background music
            elif high_score_button.is_clicked(event):
                game_state = HIGH_SCORE
            elif exit_button.is_clicked(event):
                run = False

        elif game_state == GAME_OVER:
            if play_again_button.is_clicked(event):
                game_state = PLAYING
                spaceship = Spaceship()
                waves = 1
                wave_enemy_count = 10
                enemies = [Enemy(random.choice([-ENEMY_SIZE, WIDTH]), random.randint(20, 150), waves) for _ in range(wave_enemy_count)]
                player_bullets = []
                enemy_bullets = []
                score = 0
                boss = None
                damage = 10
                start_sound.play()
                pygame.mixer.music.play(-1)
            elif exit_game_over_button.is_clicked(event):
                run = False

        elif game_state == HIGH_SCORE:
            if exit_button.is_clicked(event):
                game_state = MENU

        elif game_state == PLAYING:
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    player_bullets.append(Bullet(spaceship.x + SHIP_WIDTH//2 - BULLET_WIDTH//2, spaceship.y, -7))
                    shoot_sound.play()

    if game_state == MENU:
        title_text = title_font.render("Space Invaders", True, WHITE)
        win.blit(title_text, (WIDTH//2 - title_text.get_width()//2, HEIGHT//2 - 200))
        start_button.draw()
        high_score_button.draw()
        exit_button.draw()

    elif game_state == HIGH_SCORE:
        high_score_text = title_font.render(f"High Score: {high_score}", True, WHITE)
        win.blit(high_score_text, (WIDTH//2 - high_score_text.get_width()//2, HEIGHT//2 - 50))
        exit_button.draw()

    elif game_state == PLAYING:
        keys = pygame.key.get_pressed()
        spaceship.move(keys)
        spaceship.draw()

        if not enemies and not boss:
            if waves < 3:
                waves += 1
                wave_enemy_count += 5
                enemies = [Enemy(random.choice([-ENEMY_SIZE, WIDTH]), random.randint(20, 150), waves) for _ in range(wave_enemy_count)]
            else:
                boss = Boss()
                damage = 5

        for enemy in enemies[:]:
            enemy.move()
            enemy.draw()
            if random.random() < 0.005:
                enemy_bullets.append(Bullet(enemy.x + ENEMY_SIZE//2 - BULLET_WIDTH//2, enemy.y + ENEMY_SIZE, 4))

        if boss:
            boss.move()
            boss.draw()
            if random.random() < 0.02:
                enemy_bullets.append(Bullet(boss.x + BOSS_SIZE//2 - BULLET_WIDTH//2, boss.y + BOSS_SIZE, 5))

        for bullet in player_bullets[:]:
            bullet.move()
            bullet.draw()
            if bullet.y < 0:
                player_bullets.remove(bullet)
                continue
            for enemy in enemies[:]:
                if enemy.x < bullet.x < enemy.x + ENEMY_SIZE and enemy.y < bullet.y < enemy.y + ENEMY_SIZE:
                    enemies.remove(enemy)
                    player_bullets.remove(bullet)
                    score += 10
                    explosion_sound.play()
                    break
            if boss and boss.x < bullet.x < boss.x + BOSS_SIZE and boss.y < bullet.y < boss.y + BOSS_SIZE:
                boss.health -= 1
                player_bullets.remove(bullet)
                explosion_sound.play()
                if boss.health <= 0:
                    score += 200
                    boss = None
                    explosion_sound.play()

        for bullet in enemy_bullets[:]:
            bullet.move()
            bullet.draw()
            if spaceship.x < bullet.x < spaceship.x + SHIP_WIDTH and spaceship.y < bullet.y < spaceship.y + SHIP_HEIGHT:
                spaceship.lives -= 1
                enemy_bullets.remove(bullet)
                hit_sound.play()
                if spaceship.lives <= 0:
                    if score > high_score:
                        high_score = score
                        save_high_score(high_score)
                    game_state = GAME_OVER
                    pygame.mixer.music.stop()  # Stop background music
            if bullet.y > HEIGHT:
                enemy_bullets.remove(bullet)

        score_text = font.render(f"Score: {score}", True, WHITE)
        win.blit(score_text, (10, 10))

        lives_text = font.render(f"Lives: {spaceship.lives}", True, WHITE)
        win.blit(lives_text, (WIDTH - 130, 10))

    elif game_state == GAME_OVER:
        over_text = title_font.render("GAME OVER!", True, RED)
        win.blit(over_text, (WIDTH//2 - over_text.get_width()//2, HEIGHT//2 - 100))
        final_score_text = font.render(f"Final Score: {score}", True, WHITE)
        win.blit(final_score_text, (WIDTH//2 - final_score_text.get_width()//2, HEIGHT//2 - 50))
        play_again_button.draw()
        exit_game_over_button.draw()

    pygame.display.update()

pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html
